# Define Input Parameters

In [ ]:
BATCH_FILES: int = 150
BATCH_DIRS: int = 5

In [ ]:
f = open("group-genre.txt","r")
text = f.read()
f.close()

genre_group = {}
class_name = []
group_genre = {main_genre : [genre.strip() for genre in sub_genre.split(",")] for main_genre,sub_genre in [i.split(":") for i in text.split("\n")]}

for main_genre,sub_genre in [i.split(":") for i in text.split("\n")]:
    sub_genre = [genre.strip() for genre in sub_genre.split(",")]
    main_genre = main_genre.lower()
    for genre in sub_genre:
        genre_group[genre] = main_genre
        if main_genre not in class_name:
            class_name.append(main_genre)
            
map_class = {main_genre : num for num,main_genre in zip(range(len(class_name)),class_name)}
class_name.remove("not_use")
map_class.pop('not_use')

---

# Import Section

In [17]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, Dense, Flatten

from sklearn.model_selection import train_test_split

import math

import matplotlib.pyplot as plt

%matplotlib inline

In [14]:
classes_name = ['acoustic',
              'folk',
              'alternative/indie',
              'blues',
              'jazz',
              'classical',
              'opera',
              'club',
              'instrumental',
              'country',
              'techno',
              'electronic',
              'house',
              'hip-hop',
              'rock',
              'metal',
              'pop',
              'r&b/soul',
              'reggae',
              'latin',
              'industrial']
n_classes = len(class_name)

In [3]:
parent_dir = './data_preprocess'
X = np.load(parent_dir + "/" + 'mfccs.npy')
y = np.load(parent_dir + "/" +  'y.npy')

In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1234)

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, stratify=y_train, random_state=1234)

In [8]:
def print_class_count(y_train,y_val,y_test):
    classes_train, counts_train = np.unique(y_train, return_counts=True)
    classes_val, counts_val = np.unique(y_val, return_counts=True)
    classes_test, counts_test = np.unique(y_test, return_counts=True)
    
    print("""    train class: {}
   train count: {}
       val class: {}
      val count: {}
      test class: {}
     test count: {}
 """.format(classes_train,counts_train,classes_val,counts_val,classes_test,counts_test))

In [9]:
print_class_count(y_train, y_val, y_test)

    train class: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
   train count: [432 432 432 432 432 432 432 432 432 432 432 432 432 432 432 432 432 432
 432 432 432]
       val class: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
      val count: [144 144 144 144 144 144 144 144 144 144 144 144 144 144 144 144 144 144
 144 144 144]
      test class: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
     test count: [144 144 144 144 144 144 144 144 144 144 144 144 144 144 144 144 144 144
 144 144 144]
 


In [15]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)
X_val = X_val.reshape(X_val.shape[0],X_val.shape[1],X_val.shape[2],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2],1)

y_train = keras.utils.to_categorical(y_train, n_classes)
y_val = keras.utils.to_categorical(y_val, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

# Creating a CNN Model

In [22]:
model = Sequential([
    Conv2D(32, (3,3), activation="relu", padding="same", input_shape=X_train.shape[1:]),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),
    
    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),
    
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(n_classes, activation="softmax"),
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 40, 1200, 32)      320       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 20, 600, 32)      0         
 g2D)                                                            
                                                                 
 dropout_13 (Dropout)        (None, 20, 600, 32)       0         
                                                                 
 conv2d_13 (Conv2D)          (None, 18, 598, 64)       18496     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 9, 299, 64)       0         
 g2D)                                                            
                                                                 
 dropout_14 (Dropout)        (None, 9, 299, 64)       

# Training Model

In [24]:
batch_size = 32
epochs = 10

history = model.fit(X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val),
    shuffle=True
)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

# Plotting Loss and Accuracy Graph

In [ ]:
fig, axis = plt.subplots(1, 2)
fig.set_size_inches(12,6)

axis[0].plot(history.history['loss'])
axis[0].plot(history.history['val_loss'])
axis[0].set_title("Loss")
axis[0].legend(['Train', 'Validation'], loc='upper left')

axis[1].plot(history.history['accuracy'])
axis[1].plot(history.history['val_accuracy'])
axis[1].set_title("Accuracy")
axis[1].legend(['Train', 'Validation'], loc='upper left')